In [14]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json
import pandas


In [137]:
records = []
for line in open('../upload_900ms.log'):
    # evt = AciEventUpdate(1) SensorValues: sensor_id:15 proximity_ids:[12, 20, 24, 0, 0] proximity_rssi:[53, 55, 57, 0, 0] battery:194, accel:(37, 0, 144), status:0, valid_time:1501623900
    m = re.search('^evt = AciEventUpdate.*SensorValues.*sensor_id:(\d+) .*valid_time:(\d+)$', line)
    

    if m:
        epoch_timestamp = int(m.group(2))
        if epoch_timestamp < 200000000:
            continue
        timestamp = datetime.utcfromtimestamp(epoch_timestamp).isoformat()
        sensor_col = '%02d' % int(m.group(1))
        records.append({'ts':timestamp, sensor_col:1})
        
df = pandas.DataFrame(records)
df.set_index(['ts'])

# group by timestamp, and skip first 5 rows, as sensors are syncing up
df = df.groupby("ts").sum().iloc[4:]
df.reindex_axis(sorted(df.columns), axis=1)
pandas.set_option('display.max_columns', None)
df.tail(20) # Show layout of dataframe

,02,03,07,08,11,12,15,18,20,24,27,39,40,41,42,43,44,45,46,47,48,49,50,51,55,61
ts,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-08-11T17:57:30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T17:57:40,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T17:57:50,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T17:58:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T17:58:10,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T17:58:20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T17:58:30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T17:58:40,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2017-08-11T17:58:50,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN


In [138]:
df.shape

(72, 26)

In [139]:
# Overall participation
counts = df.sum().dropna()
overall = counts.sum() / (counts.shape[0] * df.shape[0]) * 100
print "%0.2f%% overall participation" % overall

# Individual participation
sensor_participation = counts / df.shape[0] * 100
print "-------------------------------"
print "individual sensor participation"
for (sensor_id, pct) in sensor_participation.iteritems():
    print "%2d = %6.2f%%" % (int(sensor_id), pct)

99.83% overall participation
-------------------------------
individual sensor participation
 2 = 100.00%
 3 = 100.00%
 7 =  98.61%
 8 = 100.00%
11 = 100.00%
12 = 100.00%
15 = 100.00%
18 = 100.00%
20 = 100.00%
24 = 100.00%
27 = 100.00%
39 = 100.00%
40 = 100.00%
41 = 100.00%
42 = 100.00%
43 = 100.00%
44 = 100.00%
45 =  98.61%
46 = 100.00%
47 = 100.00%
48 = 100.00%
49 = 100.00%
50 = 100.00%
51 = 100.00%
55 =  98.61%
